In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
from src.interface.data import BoosterDataModule
boost = BoosterDataModule()
boost.load_data()

use d:\Coding\learndl\learndl\data\DataSet/day.20240315.pt
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


In [1]:
# %% test a specific model
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-22 22:35:16|MOD:display     |: Model Specifics:
24-05-22 22:35:16|MOD:display     |: Start Process [Data] at Wed May 22 22:35:16 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_lgbm_ShortTest!
Callback : DynamicDataLink() , assign and unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch and event information
{'random_seed': None,
 'model_name': 'gru_lgbm_ShortTest',
 'model_mod

24-05-22 22:35:20|MOD:display     |: Finish Process [Data], Cost 3.4 Secs
24-05-22 22:35:20|MOD:display     |: Start Process [Fit] at Wed May 22 22:35:20 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


  0%|          | 0/10 [00:00<?, ?it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : -0.10938: 100%|██████████| 10/10 [00:01<00:00,  8.52it/s]
FirstBite Ep#  0 : loss  0.99389, train 0.00397, valid-0.04886, best-0.0489, lr1.3e-03
Valid Ep#  1 score : -0.08880: 100%|██████████| 10/10 [00:01<00:00,  8.99it/s]
FirstBite Ep#  1 : loss  0.98146, train 0.01947, valid 0.05836, best 0.0584, lr2.5e-03
Valid Ep#  2 score : 0.07676: 100%|██████████| 10/10 [00:01<00:00,  7.85it/s]
FirstBite Ep#  2 : loss  0.95081, train 0.04833, valid 0.13298, best 0.1330, lr3.8e-03
Valid Ep#  3 score : 0.03724: 100%|██████████| 10/10 [00:01<00:00,  7.84it/s]
FirstBite Ep#  3 : loss  0.93288, train 0.07462, valid 0.13811, best 0.1381, lr5.0e-03
24-05-22 22:36:30|MOD:display     |: gru_lgbm_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.0746 Valid 0.1381 BestVal 0.1381|Cost  1.1Min, 13.7Sec/Ep
24-05-22 22:36:30|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 1.2 Min/model, 17.6 Sec/Epoch
24-05-22 22:36:30|MOD:display     |: Start Process [Test] at Wed May 22

                    hook_name  num_calls  total_time   avg_time
10             on_train_batch         40   37.576258   0.939406
21           on_fit_model_end          1   25.732399  25.732399
27              on_test_batch        105   15.873141   0.151173
15        on_validation_batch         40    4.758962   0.118974
4          on_fit_model_start          1    2.210643   2.210643
24        on_test_model_start          1    0.643893   0.643893
25   on_test_model_type_start          3    0.054397   0.018132
17    on_validation_epoch_end          4    0.040606   0.010151
29     on_test_model_type_end          3    0.007450   0.002483
12         on_train_epoch_end          4    0.001994   0.000498
22                 on_fit_end          1    0.000000   0.000000
23              on_test_start          1    0.000000   0.000000
0          on_configure_model          1    0.000000   0.000000
19           on_fit_epoch_end          4    0.000000   0.000000
26        on_test_batch_start        105

In [ ]:
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)


In [ ]:
# prepare data
from src.interface import NetDataModule
NetDataModule.prepare_data()